움직임 크기 계산하기

1. 여덟 박자 단위로 잘린 영상 가져오기
2. 영상을 9개 구간으로 나눠서(박자마다) 포즈 값 불러오기
3. 포즈 값을 하나씩 불러 올 때마다 움직임 값 상위 3개 합쳐서 저장

이 코드는 중간에 pose estimation를 수행하는 구간이 있습니다.<br><br>
때문에 openpose 설치의 번거로움을 덜기 위해(또한 설치를 한다 하더라도 제가 openpose의 코드 중 key_point_coordinates.csv를 작성하도록 수정한 부분이 반영되어 있지 않기 때문에 이 코드는 제대로 돌지 않습니다.) 제 컴퓨터에서 8beatVid.mp4에 대해 코드를 실행시킨 결과를 보냅니다.<br><br>
아래 블록의 코드를 실행시키면 결과값이 사라지니 주의해주세요.

In [1]:
import os
import cv2
import csv

#여덟 박자 단위로 잘린 영상을 불러온다.
vidcap = cv2.VideoCapture('8beatVid.mp4')
frame_num = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
idx = 0;
count = 0;

front = [([0]*2)]*18
back = [([0]*2)]*18

value = [0]*8

first_time = True

while(vidcap.isOpened()):
    ret, image = vidcap.read()
    
    if ret == False:
        break
    
    #0/8 ~ 8/8까지 총 9개 구간에서 pose estimation 수행
    if count == (int)((frame_num-2) * idx / 8):
        cv2.imwrite('movementImage.jpg', image)
        os.system('python tf_openpose/run.py --model=cmu --resize=432x368 --image=movementImage.jpg')
        
        #front에는 이전 단계에서 back에 저장했던 값을 저장
        front = []
        for i in range(18):
            front.append(back[i])
        
        #pose estimation 수행중에 생성되는 각 관절 위치가 저장되어있는 csv 파일
        key_point_csv = open('key_point_coordinates.csv','r')
        
        back = []
        key_point_raw = []

        for line in key_point_csv:
            key_point_raw.append(line.strip('\n'))

        for i in range(0, len(key_point_raw)):
            back.append(key_point_raw[i].split(','))
        
        #18, 19행에는 pose estimation으로 나온 값중 x좌표값들의 평균값과 y좌표 값들의 최소값이 저장되어있음
        x_mean = int(back[18][0].strip())
        y_min = int(back[19][0].strip())

        #0 ~ 17행에는 pose estimation으로 나온 각 관절의 위치 값이 저장되어있음
        for i in range(0, 18):
            back[i][0] = int(back[i][0].strip())
            back[i][1] = int(back[i][1].strip())
            
            #결과 값이 0일 경우는 pose estimation에서 detection이 제대로 수행되지 않은 경우
            #이를 제외하고는 x값에서는 x의 평균 값을 빼고, y값에서는 y의 최소값을 빼서 인물의 위치를 동일하게 맞춰 포즈의 변화 정도를 정확히 측정하도록 한다.
            if back[i][0] != 0 or back[i][1] != 0 :
                back[i][0] -= x_mean
                back[i][1] -= y_min
        
        #처음 수행할때는 front 값이 모두 0으로 셋팅되어있기 때문에 front와 back의 차이를 연산하는 과정은 스킵한다.
        if first_time:
            first_time = False
            idx += 1
            continue
        
        dist = [0]*18
        
        #front와 end 사이에서 동일한 관절간의 직선 거리를 구한다. 하지만 둘,중 하나의 좌표값이 0이라면 정상적인 값이 아니기 때문에 비정상이라는 의미로 dist를 0으로 한다.
        for i in range(18):
            if (front[i][0] == 0 and front[i][1] == 0) or (back[i][0] == 0 and back[i][1] == 0):
                dist[i] = 0
            else:
                dist[i] = pow(pow((front[i][0] - back[i][0]), 2) + pow((front[i][1] - back[i][1]), 2), 0.5)
                
        dist.sort(reverse = True)
        
        #상위 세 개의 dist값을 더해서 출력한다.
        for i in range(3):
            value[idx-1] += dist[i]
        idx += 1
        
    count += 1
    
print(value)

[387.999717451925, 650.1384147330929, 403.66658652482636, 510.99307483029764, 199.24494956488223, 532.9153926603346, 560.7341358987442, 408.90424735394674]
